# setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install umap-learn

In [ ]:
import pickle
import numpy as np

# import umap
import matplotlib.pyplot as plt

# load weight mats

In [ ]:
# Define the path to your pickle file in Google Drive
file_path = '/content/drive/MyDrive/ts-1L-21M_Wdec.pkl'  # Change the path if necessary

# Load the weight matrix from the pickle file
with open(file_path, 'rb') as f:
    weight_matrix_np = pickle.load(f)

# Optionally, check the shape of the loaded weight matrix
print(weight_matrix_np.shape)

torch.Size([16384, 1024])


In [ ]:
weight_matrix_np = weight_matrix_np.detach().numpy()

In [ ]:
# Define the path to your pickle file in Google Drive
file_path = '/content/drive/MyDrive/ts-2L-33M_Wdec.pkl'  # Change the path if necessary

# Load the weight matrix from the pickle file
with open(file_path, 'rb') as f:
    weight_matrix_2 = pickle.load(f)

# Optionally, check the shape of the loaded weight matrix
print(weight_matrix_2.shape)

torch.Size([16384, 1024])


In [ ]:
weight_matrix_2 = weight_matrix_2.detach().numpy()

# SVD

In [ ]:
U1, S1, Vt1 = np.linalg.svd(weight_matrix_np)
U2, S2, Vt2 = np.linalg.svd(weight_matrix_2)

In [ ]:
l2_distance = np.linalg.norm(S1 - S2)

# Step 4: Normalize the L2 distance
frobenius_norm1 = np.linalg.norm(S1)
frobenius_norm2 = np.linalg.norm(S2)
normalized_l2_distance = l2_distance / (frobenius_norm1 + frobenius_norm2)

print("L2 distance between singular values:", l2_distance)
print("Normalized L2 distance:", normalized_l2_distance)

L2 distance between singular values: 40.600677
Normalized L2 distance: 0.1957177


# SVD on transpose

In [ ]:
weight_matrix_np.T.shape

(1024, 16384)

In [ ]:
U1, S1_T, Vt1 = np.linalg.svd(weight_matrix_np.T)
U2, S2_T, Vt2 = np.linalg.svd(weight_matrix_2.T)

In [ ]:
S1_T.shape

(1024,)

In [ ]:
S1_T

array([51.533367  , 19.960787  , 18.133106  , ...,  0.44218877,
        0.42903757,  0.26859504], dtype=float32)

In [ ]:
l2_distance = np.linalg.norm(S1_T - S2_T)

# Step 4: Normalize the L2 distance
frobenius_norm1 = np.linalg.norm(S1_T)
frobenius_norm2 = np.linalg.norm(S2_T)
normalized_l2_distance = l2_distance / (frobenius_norm1 + frobenius_norm2)

print("L2 distance between singular values:", l2_distance)
print("Normalized L2 distance:", normalized_l2_distance)

L2 distance between singular values: 40.600677
Normalized L2 distance: 0.1957177


In [ ]:
# Step 1: Compute the sum of singular values for both matrices
sum_singular_values1 = np.sum(S1_T)
sum_singular_values2 = np.sum(S2_T)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Step 2: Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

Percentage L2 distance relative to the sum of singular values: 1.69885728354048


In [ ]:
l2_distance / average_sum_singular_values

0.0169885728354048

That's 1%, which means highly similar.

# fns

In [ ]:
def compare_singular_values(S1, S2):
    # Compute the L2 distance between the singular values
    l2_distance = np.linalg.norm(S1 - S2)

    # Compute the sum of singular values for normalization
    sum_singular_values1 = np.sum(S1)
    sum_singular_values2 = np.sum(S2)
    average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

    # Calculate the percentage L2 distance
    percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

    # Output results
    print("(Singular values of the first matrix):", S1)
    print("(Singular values of the second matrix):", S2)
    print("L2 distance between singular values:", l2_distance)
    print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

In [ ]:
def normalize_singular_values(singular_values):
    """Normalize singular values by their sum."""
    # return singular_values / np.sum(singular_values)
    return singular_values / np.linalg.norm(singular_values)

def compare_normalized_singular_values(S1, S2):
    """Compare normalized singular values using L2 distance."""
    # Normalize singular values
    S1_normalized = normalize_singular_values(S1)
    S2_normalized = normalize_singular_values(S2)

    # Calculate L2 distance between normalized singular values
    l2_distance_normalized = np.linalg.norm(S1_normalized - S2_normalized)

    # Normalize the L2 distance
    frobenius_norm1 = np.linalg.norm(S1_normalized)
    frobenius_norm2 = np.linalg.norm(S2_normalized)
    normalized_l2_distance = l2_distance_normalized / (frobenius_norm1 + frobenius_norm2)

    print("Normalized Singular Values (Matrix 1):", S1_normalized)
    print("Normalized Singular Values (Matrix 2):", S2_normalized)
    print("L2 distance between normalized singular values:", l2_distance_normalized)
    print("Normalized L2 distance:", normalized_l2_distance)

In [ ]:
compare_normalized_singular_values(S1_T, S2_T)

Normalized Singular Values (Matrix 1): [0.4308114  0.16686927 0.15159012 ... 0.00369663 0.00358669 0.00224542]
Normalized Singular Values (Matrix 2): [0.6162595  0.11476967 0.09808488 ... 0.00568093 0.00560271 0.00531718]
L2 distance between normalized singular values: 0.24635288
Normalized L2 distance: 0.12317644


# compare two rand

In [ ]:
import numpy as np

def compare_svd(n, m):
    # Generate two random matrices of size n x m
    matrix1 = np.random.rand(n, m)
    matrix2 = np.random.rand(n, m)

    # Perform SVD on both matrices
    U1, S1_rand, Vt1 = np.linalg.svd(matrix1)
    U2, S2_rand, Vt2 = np.linalg.svd(matrix2)

    # Compute the L2 distance between the singular values
    l2_distance = np.linalg.norm(S1_rand - S2_rand)

    # Compute the sum of singular values for normalization
    sum_singular_values1 = np.sum(S1_rand)
    sum_singular_values2 = np.sum(S2_rand)
    average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

    # Calculate the percentage L2 distance
    percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

    # Output results
    print("S1_rand (Singular values of the first random matrix):", S1_rand)
    print("S2_rand (Singular values of the second random matrix):", S2_rand)
    print("L2 distance between singular values:", l2_distance)
    print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

# Example usage:
n = 5  # specify the number of rows
m = 4  # specify the number of columns
compare_svd(weight_matrix_2.shape[0], weight_matrix_2.shape[1])

S1_rand (Singular values of the first random matrix): [2048.34203292   46.05759418   46.04201194 ...   27.94238155   27.87871205
   27.81067192]
S2_rand (Singular values of the second random matrix): [2048.17367911   46.23345008   46.04832952 ...   27.91162767   27.88883341
   27.76678338]
L2 distance between singular values: 0.7479736770081691
Percentage L2 distance relative to the sum of singular values: 0.0018910903532818148


In [ ]:
n, m = weight_matrix_2.shape[0], weight_matrix_2.shape[1]

# Generate two random matrices of size n x m
matrix1 = np.random.rand(n, m)
matrix2 = np.random.rand(n, m)

# # Perform SVD on both matrices
U1, S1_rand, Vt1 = np.linalg.svd(matrix1)
U2, S2_rand, Vt2 = np.linalg.svd(matrix2)

In [ ]:
compare_normalized_singular_values(S1_rand, S2_rand)

Normalized Singular Values (Matrix 1): [0.86620218 0.01952138 0.01945788 ... 0.01182184 0.01180978 0.01176467]
Normalized Singular Values (Matrix 2): [0.8661393  0.01948112 0.0194627  ... 0.01182698 0.01179423 0.01175567]
L2 distance between normalized singular values: 0.00028122940580298467
Normalized L2 distance: 0.00014061470290149233


While we generally do not expect two random matrices to have highly similar singular values, the large size of the matrices (16384x1024) causes the singular values to stabilize and appear similar due to the reasons outlined above.

# compare rand to SAE

In [ ]:
compare_singular_values(S1_T, S2_rand)

(Singular values of the first matrix): [51.533367   19.960787   18.133106   ...  0.44218877  0.42903757
  0.26859504]
(Singular values of the second matrix): [2048.00037673   46.06343394   46.0198692  ...   27.96509316   27.8876412
   27.79648237]
L2 distance between singular values: 2274.8495642471826
Percentage L2 distance relative to the sum of singular values: 10.735842225955684


In [ ]:
compare_singular_values(S2_T, S2_rand)

(Singular values of the first matrix): [54.123463   10.079735    8.61438    ...  0.49893224  0.4920625
  0.46698555]
(Singular values of the second matrix): [2048.00037673   46.06343394   46.0198692  ...   27.96509316   27.8876412
   27.79648237]
L2 distance between singular values: 2286.549380387436
Percentage L2 distance relative to the sum of singular values: 11.018851051717924


In [ ]:
# from scipy import spatial
# 1 - spatial.distance.cosine(S2_T, S2_rand)

0.8916794595042503

In [ ]:
compare_normalized_singular_values(S2_T, S2_rand)

Normalized Singular Values (Matrix 1): [0.6162595  0.11476967 0.09808488 ... 0.00568093 0.00560271 0.00531718]
Normalized Singular Values (Matrix 2): [0.8661393  0.01948112 0.0194627  ... 0.01182698 0.01179423 0.01175567]
L2 distance between normalized singular values: 0.46544722790058496
Normalized L2 distance: 0.23272361395029248


In [ ]:
compare_normalized_singular_values(S1_T, S1_rand)

Normalized Singular Values (Matrix 1): [0.4308114  0.16686927 0.15159012 ... 0.00369663 0.00358669 0.00224542]
Normalized Singular Values (Matrix 2): [0.86620218 0.01952138 0.01945788 ... 0.01182184 0.01180978 0.01176467]
L2 distance between normalized singular values: 0.6919031372403384
Normalized L2 distance: 0.3459515686201692


In [ ]:
compare_normalized_singular_values(S2_T, S1_rand)

Normalized Singular Values (Matrix 1): [0.6162595  0.11476967 0.09808488 ... 0.00568093 0.00560271 0.00531718]
Normalized Singular Values (Matrix 2): [0.86620218 0.01952138 0.01945788 ... 0.01182184 0.01180978 0.01176467]
L2 distance between normalized singular values: 0.4655128513171241
Normalized L2 distance: 0.23275642565856206


# compare weight matrices of orig LLMs

In [ ]:
%%capture
%pip install transformer-lens

In [ ]:
from transformer_lens import HookedTransformer

In [ ]:
from transformers import AutoModelForCausalLM

# Load the TinyStories-1Layer-21M model
model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-1Layer-21M")

In [ ]:
# Accessing the first (and only) transformer's MLP layer weights
# Depending on the architecture, the MLP might be accessible via different paths
mlp_weights = model.transformer.h[0].mlp.c_fc.weight  # Example for GPT-like models
mlp_weights.shape

torch.Size([4096, 1024])

In [ ]:
model_2 = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-2Layers-33M")

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/323M [00:00<?, ?B/s]

In [ ]:
# Accessing the first (and only) transformer's MLP layer weights
# Depending on the architecture, the MLP might be accessible via different paths
mlp_weights_2 = model_2.transformer.h[0].mlp.c_fc.weight  # Example for GPT-like models
mlp_weights_2.shape

torch.Size([4096, 1024])

In [ ]:
U1, S1_LLM, Vt1 = np.linalg.svd(mlp_weights.detach().numpy())
U2, S2_LLM, Vt2 = np.linalg.svd(mlp_weights_2.detach().numpy())

In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S1_LLM - S2_LLM)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S1_LLM)
sum_singular_values2 = np.sum(S2_LLM)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S1_LLM)
print("(Singular values of the second matrix):", S2_LLM)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [12.483639    6.156522    6.0103292  ...  0.67446536  0.4566468
  0.32894588]
(Singular values of the second matrix): [12.764987    6.1583524   5.954615   ...  0.38923234  0.3779573
  0.36897987]
L2 distance between singular values: 3.64023
Percentage L2 distance relative to the sum of singular values: 0.17752614468019207


In [ ]:
# from scipy import spatial
# 1 - spatial.distance.cosine(S1_LLM, S2_LLM)

0.9988461785047597

In [ ]:
compare_normalized_singular_values(S1_LLM, S2_LLM)

Normalized Singular Values (Matrix 1): [0.17697017 0.08727589 0.08520344 ... 0.00956133 0.0064735  0.00466319]
Normalized Singular Values (Matrix 2): [0.18465897 0.08908705 0.08613978 ... 0.00563066 0.00546755 0.00533768]
L2 distance between normalized singular values: 0.048038624
Normalized L2 distance: 0.024019312


# LLM to rand

In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S1_LLM - S1_rand)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S1_LLM)
sum_singular_values2 = np.sum(S1_rand)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S1_LLM)
print("(Singular values of the second matrix):", S1_rand)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [12.483639    6.156522    6.0103292  ...  0.67446536  0.4566468
  0.32894588]
(Singular values of the second matrix): [2048.67570592   46.17048036   46.02030859 ...   27.96012032   27.93158454
   27.82490162]
L2 distance between singular values: 2321.350321218317
Percentage L2 distance relative to the sum of singular values: 11.149257757029565


In [ ]:
compare_normalized_singular_values(S1_LLM, S1_rand)

Normalized Singular Values (Matrix 1): [0.17697017 0.08727589 0.08520344 ... 0.00956133 0.0064735  0.00466319]
Normalized Singular Values (Matrix 2): [0.86620218 0.01952138 0.01945788 ... 0.01182184 0.01180978 0.01176467]
L2 distance between normalized singular values: 0.8605631642255557
Normalized L2 distance: 0.43028158211277784


In [ ]:
compare_normalized_singular_values(S2_LLM, S2_rand)

Normalized Singular Values (Matrix 1): [0.18465897 0.08908705 0.08613978 ... 0.00563066 0.00546755 0.00533768]
Normalized Singular Values (Matrix 2): [0.8661393  0.01948112 0.0194627  ... 0.01182698 0.01179423 0.01175567]
L2 distance between normalized singular values: 0.8588227224554329
Normalized L2 distance: 0.42941136122771645


## to MLP1

In [ ]:
mlp_weights_2b = model_2.transformer.h[1].mlp.c_fc.weight  # Example for GPT-like models
mlp_weights_2b.shape

torch.Size([4096, 1024])

In [ ]:
U2, S2_LLM_MLP1, Vt2 = np.linalg.svd(mlp_weights_2b.detach().numpy())

In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S1_LLM - S2_LLM_MLP1)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S1_LLM)
sum_singular_values2 = np.sum(S2_LLM_MLP1)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S1_LLM)
print("(Singular values of the second matrix):", S2_LLM_MLP1)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [12.483639    6.156522    6.0103292  ...  0.67446536  0.4566468
  0.32894588]
(Singular values of the second matrix): [11.038577    6.0251775   5.6920376  ...  0.56828624  0.50219357
  0.39337894]
L2 distance between singular values: 1.8857149
Percentage L2 distance relative to the sum of singular values: 0.09090751807576634


In [ ]:
S1_LLM

array([12.483639  ,  6.156522  ,  6.0103292 , ...,  0.67446536,
        0.4566468 ,  0.32894588], dtype=float32)

In [ ]:
from scipy import spatial
1 - spatial.distance.cosine(S1_LLM, S2_LLM_MLP1)

0.9997353924212166

In [ ]:
compare_normalized_singular_values(S1_LLM, S2_LLM_MLP1)

Normalized Singular Values (Matrix 1): [0.17697017 0.08727589 0.08520344 ... 0.00956133 0.0064735  0.00466319]
Normalized Singular Values (Matrix 2): [0.158688   0.08661654 0.08182739 ... 0.00816955 0.00721942 0.00565512]
L2 distance between normalized singular values: 0.023004001
Normalized L2 distance: 0.0115020005


# compare saes to LLMs

In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S1_LLM - S1_T)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S1_LLM)
sum_singular_values2 = np.sum(S1_T)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S1_LLM)
print("(Singular values of the second matrix):", S1_T)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [12.483639    6.156522    6.0103292  ...  0.67446536  0.4566468
  0.32894588]
(Singular values of the second matrix): [51.533367   19.960787   18.133106   ...  0.44218877  0.42903757
  0.26859504]
L2 distance between singular values: 60.464577
Percentage L2 distance relative to the sum of singular values: 2.4595690000021846


In [ ]:
compare_normalized_singular_values(S1_LLM, S1_T)

Normalized Singular Values (Matrix 1): [0.17697017 0.08727589 0.08520344 ... 0.00956133 0.0064735  0.00466319]
Normalized Singular Values (Matrix 2): [0.4308114  0.16686927 0.15159012 ... 0.00369663 0.00358669 0.00224542]
L2 distance between normalized singular values: 0.38446793
Normalized L2 distance: 0.19223396


In [ ]:
compare_normalized_singular_values(S2_LLM, S2_T)

Normalized Singular Values (Matrix 1): [0.18465897 0.08908705 0.08613978 ... 0.00563066 0.00546755 0.00533768]
Normalized Singular Values (Matrix 2): [0.6162595  0.11476967 0.09808488 ... 0.00568093 0.00560271 0.00531718]
L2 distance between normalized singular values: 0.49300984
Normalized L2 distance: 0.24650492


# compare to ts LLMs to gpt2 med

In [ ]:
gpt2_med = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-medium")

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [ ]:
mlp0_weights_gpt2_med = gpt2_med.transformer.h[0].mlp.c_fc.weight
mlp0_weights_gpt2_med.shape

torch.Size([1024, 4096])

In [ ]:
U2, S2_GPT2_0, Vt2 = np.linalg.svd(mlp0_weights_gpt2_med.detach().numpy())

In [ ]:
S2_GPT2_0.shape

(1024,)

In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S1_LLM - S2_GPT2_0)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S1_LLM)
sum_singular_values2 = np.sum(S2_GPT2_0)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S1_LLM)
print("(Singular values of the second matrix):", S2_GPT2_0)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [12.483639    6.156522    6.0103292  ...  0.67446536  0.4566468
  0.32894588]
(Singular values of the second matrix): [34.8373     22.285692   20.11617    ...  0.92588705  0.7650304
  0.7380823 ]
L2 distance between singular values: 138.70842
Percentage L2 distance relative to the sum of singular values: 3.331192985206716


In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S2_LLM - S2_GPT2_0)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S2_LLM)
sum_singular_values2 = np.sum(S2_GPT2_0)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S2_LLM)
print("(Singular values of the second matrix):", S2_GPT2_0)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [12.764987    6.1583524   5.954615   ...  0.38923234  0.3779573
  0.36897987]
(Singular values of the second matrix): [34.8373     22.285692   20.11617    ...  0.92588705  0.7650304
  0.7380823 ]
L2 distance between singular values: 140.22565
Percentage L2 distance relative to the sum of singular values: 3.398827863980668


In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S2_LLM_MLP1 - S2_GPT2_0)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S2_LLM_MLP1)
sum_singular_values2 = np.sum(S2_GPT2_0)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S2_LLM_MLP1)
print("(Singular values of the second matrix):", S2_GPT2_0)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [11.038577    6.0251775   5.6920376  ...  0.56828624  0.50219357
  0.39337894]
(Singular values of the second matrix): [34.8373     22.285692   20.11617    ...  0.92588705  0.7650304
  0.7380823 ]
L2 distance between singular values: 139.67618
Percentage L2 distance relative to the sum of singular values: 3.366098966012683


In [ ]:
compare_normalized_singular_values(S2_LLM_MLP1, S2_GPT2_0)

Normalized Singular Values (Matrix 1): [0.158688   0.08661654 0.08182739 ... 0.00816955 0.00721942 0.00565512]
Normalized Singular Values (Matrix 2): [0.16676885 0.10668334 0.09629767 ... 0.00443229 0.00366226 0.00353326]
L2 distance between normalized singular values: 0.08103228
Normalized L2 distance: 0.040516146


In [ ]:
compare_normalized_singular_values(S2_LLM, S2_GPT2_0)

Normalized Singular Values (Matrix 1): [0.18465897 0.08908705 0.08613978 ... 0.00563066 0.00546755 0.00533768]
Normalized Singular Values (Matrix 2): [0.16676885 0.10668334 0.09629767 ... 0.00443229 0.00366226 0.00353326]
L2 distance between normalized singular values: 0.09416299
Normalized L2 distance: 0.047081497


In [ ]:
compare_normalized_singular_values(S1_LLM, S2_GPT2_0)

Normalized Singular Values (Matrix 1): [0.17697017 0.08727589 0.08520344 ... 0.00956133 0.0064735  0.00466319]
Normalized Singular Values (Matrix 2): [0.16676885 0.10668334 0.09629767 ... 0.00443229 0.00366226 0.00353326]
L2 distance between normalized singular values: 0.08154233
Normalized L2 distance: 0.040771164


# to mid layer of GPT2_med

In [ ]:
mlp7_weights_gpt2_med = gpt2_med.transformer.h[7].mlp.c_fc.weight
mlp7_weights_gpt2_med.shape

torch.Size([1024, 4096])

In [ ]:
U2, S2_GPT2_7, Vt2 = np.linalg.svd(mlp7_weights_gpt2_med.detach().numpy())

In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S2_LLM_MLP1 - S2_GPT2_7)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S2_LLM_MLP1)
sum_singular_values2 = np.sum(S2_GPT2_7)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S2_LLM_MLP1)
print("(Singular values of the second matrix):", S2_GPT2_7)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [11.038577    6.0251775   5.6920376  ...  0.56828624  0.50219357
  0.39337894]
(Singular values of the second matrix): [38.283348  19.853462  18.42683   ...  1.3392195  1.2193433  1.1025848]
L2 distance between singular values: 146.5148
Percentage L2 distance relative to the sum of singular values: 3.5015221748525756


In [ ]:
compare_normalized_singular_values(S2_LLM_MLP1, S2_GPT2_7)

Normalized Singular Values (Matrix 1): [0.158688   0.08661654 0.08182739 ... 0.00816955 0.00721942 0.00565512]
Normalized Singular Values (Matrix 2): [0.17724231 0.09191655 0.08531161 ... 0.00620025 0.00564525 0.00510469]
L2 distance between normalized singular values: 0.039959364
Normalized L2 distance: 0.019979682


In [ ]:
# Compute the L2 distance between the singular values
l2_distance = np.linalg.norm(S2_GPT2_0 - S2_GPT2_7)

# Compute the sum of singular values for normalization
sum_singular_values1 = np.sum(S2_GPT2_0)
sum_singular_values2 = np.sum(S2_GPT2_7)
average_sum_singular_values = (sum_singular_values1 + sum_singular_values2) / 2

# Calculate the percentage L2 distance
percentage_l2_distance = (l2_distance / average_sum_singular_values) * 100

# Output results
print("(Singular values of the first matrix):", S2_GPT2_0)
print("(Singular values of the second matrix):", S2_GPT2_7)
print("L2 distance between singular values:", l2_distance)
print("Percentage L2 distance relative to the sum of singular values:", percentage_l2_distance)

(Singular values of the first matrix): [34.8373     22.285692   20.11617    ...  0.92588705  0.7650304
  0.7380823 ]
(Singular values of the second matrix): [38.283348  19.853462  18.42683   ...  1.3392195  1.2193433  1.1025848]
L2 distance between singular values: 22.271751
Percentage L2 distance relative to the sum of singular values: 0.3549893561672941


In [ ]:
compare_normalized_singular_values(S2_GPT2_0, S2_GPT2_7)

Normalized Singular Values (Matrix 1): [0.16676885 0.10668334 0.09629767 ... 0.00443229 0.00366226 0.00353326]
Normalized Singular Values (Matrix 2): [0.17724231 0.09191655 0.08531161 ... 0.00620025 0.00564525 0.00510469]
L2 distance between normalized singular values: 0.09938153
Normalized L2 distance: 0.049690764


# pythia

In [ ]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer

model_pythia410 = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-410m",
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

In [ ]:
model_pythia410.gpt_neox.layers[0].mlp

GPTNeoXMLP(
  (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
  (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  (act): GELUActivation()
)

In [ ]:
# Accessing the first (and only) transformer's MLP layer weights
# Depending on the architecture, the MLP might be accessible via different paths
model_pythia410_weights_mlp0 = model_pythia410.gpt_neox.layers[0].mlp.dense_4h_to_h.weight  # Example for GPT-like models
model_pythia410_weights_mlp0.shape

torch.Size([1024, 4096])

In [ ]:
U1, S_pythia410_mlp0, Vt1 = np.linalg.svd(model_pythia410_weights_mlp0.detach().numpy())

In [ ]:
compare_singular_values(S_pythia410_mlp0, S2_rand)

(Singular values of the first matrix): [3.5718095  2.2339969  1.9763415  ... 0.40511042 0.31147727 0.08539316]
(Singular values of the second matrix): [2048.00037673   46.06343394   46.0198692  ...   27.96509316   27.8876412
   27.79648237]
L2 distance between singular values: 2343.3803602484672
Percentage L2 distance relative to the sum of singular values: 11.51450891317988


In [ ]:
compare_singular_values(S_pythia410_mlp0, S1_LLM)

(Singular values of the first matrix): [3.5718095  2.2339969  1.9763415  ... 0.40511042 0.31147727 0.08539316]
(Singular values of the second matrix): [12.483639    6.156522    6.0103292  ...  0.67446536  0.4566468
  0.32894588]
L2 distance between singular values: 35.51397
Percentage L2 distance relative to the sum of singular values: 2.1914267271554673


In [ ]:
compare_singular_values(S_pythia410_mlp0, S1_T)

(Singular values of the first matrix): [3.5718095  2.2339969  1.9763415  ... 0.40511042 0.31147727 0.08539316]
(Singular values of the second matrix): [51.533367   19.960787   18.133106   ...  0.44218877  0.42903757
  0.26859504]
L2 distance between singular values: 91.21451
Percentage L2 distance relative to the sum of singular values: 4.5832401785749886


In [ ]:
compare_singular_values(S_pythia410_mlp0, S2_GPT2_0)

(Singular values of the first matrix): [3.5718095  2.2339969  1.9763415  ... 0.40511042 0.31147727 0.08539316]
(Singular values of the second matrix): [34.8373     22.285692   20.11617    ...  0.92588705  0.7650304
  0.7380823 ]
L2 distance between singular values: 172.8472
Percentage L2 distance relative to the sum of singular values: 4.676903828670911


In [ ]:
compare_singular_values(S_pythia410_mlp0, S2_GPT2_7)

(Singular values of the first matrix): [3.5718095  2.2339969  1.9763415  ... 0.40511042 0.31147727 0.08539316]
(Singular values of the second matrix): [38.283348  19.853462  18.42683   ...  1.3392195  1.2193433  1.1025848]
L2 distance between singular values: 180.53325
Percentage L2 distance relative to the sum of singular values: 4.839278992349839


In [ ]:
model_pythia410_weights_mlp17 = model_pythia410.gpt_neox.layers[17].mlp.dense_4h_to_h.weight  # Example for GPT-like models
model_pythia410_weights_mlp17.shape

torch.Size([1024, 4096])

In [ ]:
U1, S_pythia410_mlp17, Vt1 = np.linalg.svd(model_pythia410_weights_mlp17.detach().numpy())

In [ ]:
compare_singular_values(S_pythia410_mlp0, S_pythia410_mlp17)

(Singular values of the first matrix): [3.5718095  2.2339969  1.9763415  ... 0.40511042 0.31147727 0.08539316]
(Singular values of the second matrix): [4.483849   3.7432733  3.6153734  ... 0.34852073 0.30791458 0.09956443]
L2 distance between singular values: 10.75429
Percentage L2 distance relative to the sum of singular values: 0.8461361801275343


In [ ]:
compare_singular_values(S_pythia410_mlp17, S1_LLM)

(Singular values of the first matrix): [4.483849   3.7432733  3.6153734  ... 0.34852073 0.30791458 0.09956443]
(Singular values of the second matrix): [12.483639    6.156522    6.0103292  ...  0.67446536  0.4566468
  0.32894588]
L2 distance between singular values: 25.673023
Percentage L2 distance relative to the sum of singular values: 1.476179617287453


In [ ]:
compare_singular_values(S_pythia410_mlp17, S1_T)

(Singular values of the first matrix): [4.483849   3.7432733  3.6153734  ... 0.34852073 0.30791458 0.09956443]
(Singular values of the second matrix): [51.533367   19.960787   18.133106   ...  0.44218877  0.42903757
  0.26859504]
L2 distance between singular values: 82.25396
Percentage L2 distance relative to the sum of singular values: 3.9006180646133806


In [ ]:
compare_singular_values(S_pythia410_mlp17, S2_GPT2_0)

(Singular values of the first matrix): [4.483849   3.7432733  3.6153734  ... 0.34852073 0.30791458 0.09956443]
(Singular values of the second matrix): [34.8373     22.285692   20.11617    ...  0.92588705  0.7650304
  0.7380823 ]
L2 distance between singular values: 163.33432
Percentage L2 distance relative to the sum of singular values: 4.28212612089261


In [ ]:
compare_normalized_singular_values(S_pythia410_mlp17, S2_GPT2_0)

Normalized Singular Values (Matrix 1): [0.0978341  0.08167531 0.07888463 ... 0.00760445 0.00671846 0.00217242]
Normalized Singular Values (Matrix 2): [0.16676885 0.10668334 0.09629767 ... 0.00443229 0.00366226 0.00353326]
L2 distance between normalized singular values: 0.095897116
Normalized L2 distance: 0.047948558


In [ ]:
compare_normalized_singular_values(S_pythia410_mlp17, S2_rand)

Normalized Singular Values (Matrix 1): [0.0978341  0.08167531 0.07888463 ... 0.00760445 0.00671846 0.00217242]
Normalized Singular Values (Matrix 2): [0.8661393  0.01948112 0.0194627  ... 0.01182698 0.01179423 0.01175567]
L2 distance between normalized singular values: 0.9246961239314375
Normalized L2 distance: 0.46234806196571876


# load sae f actvs

In [ ]:
file_path = '/content/drive/MyDrive/fActs_ts_1L_21M_anySamps_v1.pkl'
with open(file_path, 'rb') as f:
    feature_acts_model_A = pickle.load(f)

In [ ]:
file_path = '/content/drive/MyDrive/fActs_ts_2L_33M_anySamps_v1.pkl'
with open(file_path, 'rb') as f:
    feature_acts_model_B = pickle.load(f)

In [ ]:
feature_acts_model_B.shape

torch.Size([500, 128, 16384])

In [ ]:
first_dim_reshaped = feature_acts_model_A.shape[0] * feature_acts_model_A.shape[1]
reshaped_activations_A = feature_acts_model_A.reshape(first_dim_reshaped, feature_acts_model_A.shape[-1]).cpu()
reshaped_activations_B = feature_acts_model_B.reshape(first_dim_reshaped, feature_acts_model_B.shape[-1]).cpu()

In [ ]:
reshaped_activations_B.shape

torch.Size([64000, 16384])